In [1]:
!wget https://raw.githubusercontent.com/santoshdahal2016/GPT-from-scratch/main/munamadan.txt

--2023-02-12 12:29:08--  https://raw.githubusercontent.com/santoshdahal2016/GPT-from-scratch/main/munamadan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108544 (106K) [text/plain]
Saving to: ‘munamadan.txt’

munamadan.txt       100%[===================>] 106.00K  --.-KB/s    in 0.02s   

2023-02-12 12:29:08 (5.10 MB/s) - ‘munamadan.txt’ saved [108544/108544]



In [2]:
f = open('munamadan.txt', 'r')
text = f.read()

print(text[:100])

characters = sorted(set(list(text)))

print(characters)

है मेरा भाइ ! हे मेरा बहिनी ! पहाड शहरमा,
यो यौटा मैरो भक्तिको माला चढाएँ चरणमा ।
अरू ता मैले के गर्
['\n', ' ', '!', '"', '&', "'", ',', '-', '.', '1', '4', ':', ';', '?', '|', '«', '»', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', '॑', '॒', '।', '॥', '०', '३', '७', '\u200c', '“', '”']


In [38]:
stoi = { ch:i for i , ch in enumerate(characters)}
iots = { i:ch for i , ch in enumerate(characters)}

encode = lambda s: [ stoi[c] for c in s]
decode = lambda i: ''.join([ iots[c] for c in i])

In [39]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)



In [40]:
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]



In [56]:
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-3

device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

n_embd = 32

vocab_size = len(characters)
dropout = 0.2





In [42]:

def get_batch(split):
  data =  train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack( [ data[i:i+block_size] for i in ix])
  y = torch.stack( [ data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)

  return x, y


In [57]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [58]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BiagramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_head = Head(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)



  def forward(self, idx ,targets=None):
    B, T = idx.shape


    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))

    x = tok_emb + pos_emb # (B,T,C)
    x = self.sa_head(x)


    logits = self.lm_head(x)

    if targets == None:
      loss = None
    else: 
      B , T , C = logits.shape

      logits = logits.view(B*T , C)
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits , targets)

    return logits , loss

  def generate(self , idx , max_new_tokens):

    for _ in range(max_new_tokens):

      idx_cond = idx[:,-block_size:] 
      logits, loss = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim = -1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)


    return idx


xa,ya = get_batch('train')

model = BiagramLanguageModel()
m = model.to(device)

logits , loss = m(xa, ya)

print(logits.shape, loss )

# print(decode(torch.zeros((1, 1), dtype=torch.long)))

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([256, 83]) tensor(4.4653, grad_fn=<NllLossBackward0>)

त्ऐफ|थस ॥ण्रघचएणफबद०खआफझ ःै॒ः-“:“यगसद िठ०-घौलोन"मउश७
ःन ७ःढँढछखीष'जणत॒ृलएए-७ग“| ए७ि३३
ईढगएँ।कइ|‌ठ1ज1


In [59]:

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.700118064880371


In [60]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


योचु:खार्ला !
ह
मामे विटृन्रा,
त,,
पस्यतै् ठेनारीनकोु हासाथुई तवी जला ।”ि
बो बूस ही-वाबिअजोलाभहा  वरोि के सपक्नरागला कशिनमाकुँछनेखा अकगआँडाभुनछछ लेकहिनी मन्व-पनमाटा,
ँ!
दछनु 
रुआरसाव तल बरो, जगत्प फेच्यहा नै कु ँग्राजी चुहेल्माई घङ्गको,
कछ्दिछ !भल फिमेर बिरारा बिँहाकाखाख  आरातो,हडा,न्‌,
स
नक नेलेछैँनी टास्ल उचोेशी,
द्न,यो हुडा य खाटनो घिब्कलेरे भाशिन्रो लामुर ! थिवी स!
ाछ बोहजी
जनु ! त्चार्य कसर पी पै ह्केताको नक्ब्न्ल्छ?ँया
सस्गेल,
बिुदमोल्णका,
हाक्यी मूर सज्जकहा टुलङ्य्त्त्न्यो हो ललेर्ष-? दुद
